# Projekt - Dynamic Inverse Kinematics

## Opis zagadnienia

Celem projektu jest rozwiązanie problemu Inverse Kinematics w dynamicznie zmieniającym się środowisku. W klasycznej wersji problemu dane jest ramię złożone z pewnej liczby segmentów. Celem jest takie dobranie kątów między segmentami, aby koniec ostatniego segmentu znalazł się w zadanym punkcie. W moim projekcie rozważam wersję problemu, w której obecne są dodatkowo prostokątne przeszkody. Co więcej, zarówno położenie przeszkód jak i celu może zmieniać się wraz z czasem.

## Opis metody

### Przestrzeń poszukiwań

Przestrzenią poszukiwań jest zbiór wektorów o wartościach rzeczywistych z zadanych przedziałów. Wartości wektora reprezentują kąty w radianach między kolejbymi segmentami, a przedziały pozwalają na ograniczenie zakresu ruchu danego segmentu.

### Funkcja celu

Jako funkcję celu stosuję odległość euklidesową końca ostatniego segmentu od celu, podniesioną do kwadratu.

### Funkcje ograniczeń

Dla każdej przeszkody definiuję osobną funkcję ograniczeń, pozwalającą oszacować, w jakim stopniu ramię "nachodzi" na daną przeszkodę. Dla każdej kolizji liczona jest wartość "kary", liczonej poprzez szacowanie stosunku pól figur, na które ramię dzieli daną przeszkodę.

TODO: rysunek

### Użyty algorytm

Do rozwiązania problemu stosuję algorytm IDEA w wersji dostosowanej do rozwiązywania problemów dynamicznych. W kolejnych krokach ewolucja jest rozpoczynana z populacją z poprzedniego kroku, w której część najsłabszych osobników zastąpiono losowymi imigrantami.

### Operatory ewolucyjne

Używane są operatory *simulated binary crossover* oraz *polynomial mutation*. W przypadku mutacji wprowadziłem dodatkową możliwość multiplikatywnego zmniejszania prawdopodobieństwa mutacji dla wartości reprezentujących kąty bliżej podstawy ramienia.